# Dataset Creation - TRsim Experiment 3

R = amount of starting configurations with B boxes

B = amount of boxes in the room at the start

## Step 1: Create the foundation files

Within the folder data/scene_datasets/TRsim_exp3/stages/:

- (1x) TRsim_room.glb: floor + walls + ceiling mesh.

- (Rx) TRsim.ply: export the room, as well as the boxes.

- (1x) TRsim.house: .house file describing the room geometry, and the box locations.

Within the folder data/scene_datasets/TRsim_exp3/objects/:

- (1x) TRsim_box.glb: box mesh.

And:

- (1x) TRsim_material_config.json: Acoustic properties file

## Step 1: Modify the _semantic.ply

Modify the .ply file to change the last x vertices (boxes) to different locations, as well as adding the right material "object_id". Save in the format "~/ss/data/scene_datasets/TRsim_exp3/stages/TRsim_room{n}\_{amount of boxes removed}\_{ids of boxes removed}_semantic.ply". 

## Step 2: Create ground truth label file

The label is saved in the filename, as it corresponds to the amount of boxes removed.

## Step 3: Run SoundSpaces 2.0

Load the file, rename to "treated_room_object_semantic.ply", run simulation, and save as "N.wav".

## Step 4: Concatenate mic channels

For now, the simulations are saved per mic. For the multi-channel cases, these files need to be concatenated into one "deconvolved.npy".

--------------------------------------------------------------------------------------------------------------------------------------------

In [2]:
import numpy as np
import os

from plyfile import PlyData, PlyElement

In [3]:
BASE_DIR = "/home/student/ss"
SIM_DIR = os.path.join(BASE_DIR, "data/scene_datasets/TRsim_exp3")
STAGES_DIR = os.path.join(BASE_DIR, SIM_DIR, "stages")
print(STAGES_DIR)

/home/student/ss/data/scene_datasets/TRsim_exp3/stages


### Step 1: Modify the _semantic.ply

Save with the name "~/ss/data/scene_datasets/TRsim_exp3/stages/TRsim_room{n}\_{amount of boxes removed}\_{ids of boxes removed}_semantic.ply"

In [53]:
import itertools

def create_empty_ply(base_ply):
    base_verts = np.array(base_ply['vertex'].data) # (40,), where the first 33 correspond to the room, the last 8 to the box

    verts_dtype = [
        ('x', 'f4'), ('y', 'f4'), ('z', 'f4'),
        ('red', 'u1'), ('green', 'u1'), ('blue', 'u1'),
        ('object_id', 'i4')] # remove the alpha, add the object_id
    
    new_verts = np.empty(len(base_verts), dtype=verts_dtype)

    new_verts['x'] = base_verts['x'] # last 8 will be overwritten
    new_verts['y'] = base_verts['y'] # last 8 will be overwritten
    new_verts['z'] = base_verts['z'] # does not change between measurement locations (vertex order does not change)
    new_verts['red'] = base_verts['red'] # same
    new_verts['green'] = base_verts['green'] # same
    new_verts['blue'] = base_verts['blue'] # same

    for v in range(len(new_verts)): # set all materials, does not change between measurement locations (vertex order does not change)
        if new_verts['red'][v] == 254:
            new_verts['object_id'][v] = 1 # wall material
        elif new_verts['green'][v] == 254:
            new_verts['object_id'][v] = 0 # floor material
        elif new_verts['blue'][v] == 254:
            new_verts['object_id'][v] = 2 # ceiling material
        else:
            new_verts['object_id'][v] = 3 # object material        
        
    faces = base_ply['face'] # unchanged, just sets which vertice ids correspond to which face (and vertex order does not change)
    
    return new_verts, faces

def remove_boxes_from_ply(new_verts, faces, boxes_to_remove):
    """
    Remove multiple boxes at once.
    boxes_to_remove: list of integers, the box indices to remove
    """
    # Step 1: find all box vertices
    is_box_vertex = new_verts['object_id'] == 3
    box_vertex_indices_all = np.where(is_box_vertex)[0]

    # Step 2: select the vertices corresponding to the boxes to remove
    box_vertex_indices_to_remove = []
    for box in boxes_to_remove:
        start = box * 8
        end = start + 8
        box_vertex_indices_to_remove.extend(box_vertex_indices_all[start:end])
    box_vertex_indices_to_remove = np.array(box_vertex_indices_to_remove)

    # Step 3: remove faces referencing these vertices
    faces_data = np.array(faces.data)
    keep_face_mask = []
    for face in faces_data:
        vertex_ids = face['vertex_indices']
        if not any(v in box_vertex_indices_to_remove for v in vertex_ids):
            keep_face_mask.append(True)
        else:
            keep_face_mask.append(False)
    keep_face_mask = np.array(keep_face_mask)
    remaining_faces_data = faces_data[keep_face_mask]

    # Step 4: remove the vertices
    keep_vertex_mask = np.ones(len(new_verts), dtype=bool)
    keep_vertex_mask[box_vertex_indices_to_remove] = False
    remaining_verts = new_verts[keep_vertex_mask]

    # Step 5: remap face indices
    old_to_new = np.full(len(new_verts), -1, dtype=int)
    old_to_new[np.where(keep_vertex_mask)[0]] = np.arange(len(remaining_verts))
    for i, face in enumerate(remaining_faces_data):
        remaining_faces_data[i]['vertex_indices'] = [old_to_new[v] for v in face['vertex_indices']]

    remaining_faces = PlyElement.describe(remaining_faces_data, 'face')
    return remaining_verts, remaining_faces

def save_new_ply(mod_verts, mod_faces, boxes_removed, filename_prefix='TRsim_room_', mode='binary'):
    num_removed = len(boxes_removed)
    boxes_str = "".join(map(str, boxes_removed))
    filename = f"{filename_prefix}{num_removed}_{boxes_str}"
    
    if mode == 'binary':
        ply = PlyData([PlyElement.describe(mod_verts, 'vertex'), mod_faces], byte_order='<')
        ply.write(os.path.join(STAGES_DIR, filename + "_semantic.ply"))
    elif mode == 'ascii':
        ply = PlyData([PlyElement.describe(mod_verts, 'vertex'), mod_faces], text=True)
        ply.write(os.path.join(STAGES_DIR, filename + "_ascii.ply"))
    else:
        raise Exception("Mode must be 'binary' or 'ascii'")

In [54]:
# Parameters
B = 10  # number of boxes
R = [1]  # list of room indices

labels = []

for r in R:
    base_ply = PlyData.read(os.path.join(STAGES_DIR, "base", f"TRsim_room{r}.ply"))
    new_verts, faces = create_empty_ply(base_ply)

    save_new_ply(new_verts, faces, [], filename_prefix=f'TRsim_room{r}_')
    labels.append(0)

    # Generate all combinations of boxes to remove (from 1 to B)
    for num_to_remove in range(1, B + 1):
        for boxes_to_remove in itertools.combinations(range(B), num_to_remove):
            remaining_verts, remaining_faces = remove_boxes_from_ply(new_verts, faces, list(boxes_to_remove))
            save_new_ply(remaining_verts, remaining_faces, boxes_to_remove, filename_prefix=f'TRsim_room{r}_')
            labels.append(len(boxes_to_remove))

np.save(os.path.join(BASE_DIR, "data/output/TRsim_exp3/try1", f"labels_{len(R)}rooms_{str(B)}boxes.npy"), np.array(labels))

In [57]:
assert len(labels) == 2**B # amount of possible combinations, including the "no_box_removed" and "all_boxes_removed" situation

### Step 2: Create a ground truth label file

This step is merged with the one before, as this is more efficient.

### Step 3: Run SoundSpaces 2.0

Run Docker simsetup container with: 

(maybe: '''docker rm simsetup''')

'''docker run -it --gpus all --name simsetup -v ~/ss/examples:/sound-spaces/examples -v ~/ss/data:/sound-spaces/data soundspaces:U20cudaglheadless /bin/bash'''

'''cd sound-spaces'''

'''python examples/TRsim.py'''

### Step 4: Concatenate mic channels

In [23]:
import numpy as np
import os

C = 1
R = 50000
N = 10
S = 48000

TRSIM = 'TRsim_exp3'
FOLDER = 'try1'

deconv = np.zeros((N, 10, S))

if C == 1:
    mics = [0]
elif C == 2:
    mics = [0, 6]
elif C == 4:
    mics = [0, 5, 6, 9]
elif C == 10:
    mics = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

for n in mics:
    mic_rirs = np.load(BASE_DIR + f"/data/output/{TRSIM}/{FOLDER}/deconvolved_{N}_{R}_mic{n}.npy")
    deconv[:, n, :] = mic_rirs.squeeze(1)

output = BASE_DIR + f"/data/output/{TRSIM}/{FOLDER}/deconvolved_{N}_{R}_{C}.npy"
np.save(output, deconv)

In [24]:
data_folder_server = f"menthe@10.201.224.12:/scsi_two/Menthe/{TRSIM}/"
print(f"rsync -avz {output} {data_folder_server}")

rsync -avz /home/student/ss/data/output/TRsim_exp3/try1/deconvolved_10_50000_1.npy menthe@10.201.224.12:/scsi_two/Menthe/TRsim_exp3/


In [25]:
print("/scsi_two/Menthe/TRsim_human/"+f"deconvolved_{N}_{R}_{C}.npy")

/scsi_two/Menthe/TRsim_human/deconvolved_10_50000_1.npy
